In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
#data.head()

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [7]:
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
als = ALS(maxIter=10, regParam=0.05, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [9]:
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1.0| 5|0.41839957|
 31| 1.0| 0| 1.0386931|
 85| 3.0| 1|0.56742233|
 85| 1.0| 13| 1.617557|
 85| 2.0| 20| 1.5173216|
 53| 1.0| 12| 1.239668|
 53| 1.0| 9| 3.4616032|
 53| 5.0| 21| 4.156772|
 78| 1.0| 27|0.86097014|
 78| 1.0| 12| 0.9887927|
 78| 1.0| 20|0.94337314|
 78| 1.0| 19| 0.754539|
 78| 1.0| 11| 0.7657511|
 34| 3.0| 3|0.73285216|
 34| 4.0| 2| 1.7945105|
 81| 5.0| 28| 1.1159681|
 81| 3.0| 26| 2.2087343|
 81| 1.0| 22|0.95255566|
 81| 4.0| 11| 1.3425437|
 28| 3.0| 1| 2.1796038|
+-------+------+------+----------+
only showing top 20 rows

In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.0989179196836973

In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 12| 11|
 21| 11|
 23| 11|
 27| 11|
 30| 11|
 32| 11|
 36| 11|
 38| 11|
 39| 11|
 43| 11|
 45| 11|
 47| 11|
 69| 11|
 78| 11|
 81| 11|
 90| 11|
 99| 11|
+-------+------+

In [14]:
reccomendations = model.transform(single_user)

In [15]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
movieId|userId|prediction|
+-------+------+----------+
 27| 11| 4.705982|
 32| 11| 3.287994|
 39| 11| 2.4455369|
 30| 11| 1.9323738|
 38| 11| 1.7097341|
 23| 11| 1.6459142|
 90| 11| 1.6073815|
 69| 11| 1.5596935|
 43| 11| 1.5399317|
 81| 11| 1.3425437|
 45| 11| 1.2005978|
 12| 11| 1.0190052|
 36| 11| 0.991805|
 99| 11| 0.8430539|
 78| 11| 0.7657511|
 47| 11|0.37372324|
 21| 11|0.25804684|
+-------+------+----------+

In [16]:
#Printing Movie name with Perticular MovieID
data2 = spark.read.csv('/FileStore/tables/movies.csv',inferSchema=True,header=True)

In [17]:
type(data2)

Out[17]: pyspark.sql.dataframe.DataFrame

In [18]:
type(data2['movieId'])

Out[23]: pyspark.sql.column.Column

In [19]:
data2.createOrReplaceTempView('Mnames')


In [20]:
sql_results=spark.sql('select * from Mnames Where movieId==27')

In [21]:
print(sql_results.show())

+-------+-------------------+--------------+
movieId| title| genres|
+-------+-------------------+--------------+
 27|Now and Then (1995)|Children|Drama|
+-------+-------------------+--------------+

None